In [1]:
%matplotlib qtagg
from platypus_eventer.analysis import read_events, identify_glitch, deglitch, process_events
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import differential_evolution

In [2]:
e = read_events("DAQ_2025-08-07T16-14-30")
f_f, t_f, f_v, t_v, volts = process_events(e)

In [3]:
plt.figure()
plt.plot(np.diff(t_f));
plt.show()

In [4]:
f_f, t_f, f_v, t_v, volts = deglitch(f_f, t_f, f_v, t_v, volts, 25)

plt.figure()
plt.plot(np.diff(t_f))

In [5]:
frame_frequency = np.mean(1 / np.diff(t_f))
_period = 1 / frame_frequency
_sample_frame_map = np.searchsorted(f_f, f_v)
f_sample_frac = (t_v - t_f[_sample_frame_map]) / _period
f_sample_frac += f_v

In [6]:
def wave(x, p):
    offset, amplitude, f0, osc_period = p
    return offset + amplitude * np.sin(phase(x, p))


def phase(x, p):
    x = np.asarray(x, np.float64)
    offset, amplitude, f0, osc_period = p
    return 2 * np.pi * (x - f0) / osc_period


def chi2(p, *args):
    x = args[0]
    y = args[1]
    ideal = wave(x, p)
    return np.sum((y - ideal) ** 2)

In [9]:
offset = np.mean(volts)
oscillation_period = 25

bounds = [
    (0.9 * offset, 1.1 * offset),
    (0.01, 10),
    (0, oscillation_period),
    (0.9 * oscillation_period, 1.1 * oscillation_period),
]
res = differential_evolution(
        chi2,
        bounds=bounds,
        args=(f_sample_frac, volts),
        # workers=-1,
        # polish=False
)

In [12]:
N = f_sample_frac[0]
plt.figure()
plt.plot(f_sample_frac, volts)
plt.plot(f_sample_frac, wave(f_sample_frac, res.x))